# A solution for NSUCRYPTO 2020
- Problem 2: _Stairs-Box_
- By: _ndh_

## Notation 

Let $M_A$, $M_B$ denote the $6 \times 6$ matrices corresponding to the linear maps $A$ and $B$: $A(v) = M_Av$ and $B(v) = M_Bv$. 

Also, let $x$ be the input, $t$ be the output and $y$, $z$ be the intermediate values of the function $S$ as shown in Figure 2.1.

_Figure 2.1_


When a value $v$ is treated as a vector in $\mathbb{F}_2^6$, its components are denoted as $v_i$: $v = (v_1, v_2, ..., v_6)$. When treated as an element of $\mathbb{Z}_{64}$ (the ring of integers modulo $64$), $v = \sum_{i=1}^{6}{v_i2^{6-i}}$ ($v_1$ as the most significant bit, $v_6$ as the least significant bit).

Since $X$ is a function over $\mathbb{Z}_{64}$, we can restrict its domain to $\mathbb{Z}_{32}$, $\mathbb{Z}_{16}$, ..., $\mathbb{Z}_2$ and denote $X$ over these restricted domains as $X_{32}$, $X_{16}$, ..., $X_2$ respectively. More specifically, $X_{32}(y) = X(y) \text{ mod } 32$, $X_{16}(y) = X(y) \text{ mod } 16$ and so on. Please note that $X_{64}$ is an alias of $X$, and $X_{2^i}$ only uses $i$ least significant bits of $y$, as shown in Figure 2.2.

_Figure 2.2_


For a matrix $M$, we use $M_{[i:]}$ to denote the submatrix of $M$ constructed by removing from $M$ every row before row $i$, starting from $1$. For example, if $M = I_6$ (the identity matrix of size $6 \times 6$), then $M_{[5:]} = \begin{bmatrix} 0 & 0 & 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 0 & 0 & 1 \end{bmatrix}$.

## The idea

Firstly, observe that $A, B, X$ are all invertible functions since $S$ is invertible ($S$ is a permutation). Hence, $M_A$ and $M_B$ are invertible matrices.

One approach to solve the problem is to solve for $X_2$ first, then $X_4$, then $X_8$, ... and finally $X = X_{64}$. Solving for each $X_{2^i}$ ($i \in \{1, 2, ..., 6\}$) can be done by brute-forcing $M_{B[7-i:]}$ (the submatrix of $M_B$ that determines the $i$ least significant bits of $y$ from $x$) and $M_{A[7-i:]}^{-1}$ (the submatrix of $M_A^{-1}$ that determines the $i$ least significant bits of $z$ from $t$) until $X_{2^i}$ can be derived from the known function $S$. Since $M_{B[7-(i-1):]}$ and $M_{A[7-(i-1):]}^{-1}$ have been known in advance (the step in which we solves for $X_{2^{i-1}}$ has been completed), this approach only requires just two-row brute-forcing (one for each matrix) at each step.

Please note that, using this approach, $X$ can be any function that satisfies the "higher input bits do not affect lower output bits" rule. In order to reduce the complexity, we only keep the functions that can be represented as: $X(y) = ay + b \text{ mod } 64$ for some $a, b \in \mathbb{Z}_{64}$, since $X$ should be a "short arithmetic expression modulo 64".

## The implementation

Let's implementation the above idea using Python 3. Here's the original function $S$ given as a list:

In [1]:
S = [13, 18, 20, 55, 23, 24, 34, 1, 62, 49, 11, 40, 36, 59, 61, 30,
     33, 46, 56, 27, 41, 52, 14, 45, 0, 29, 39, 4, 8, 7, 17, 50,
     2, 54, 12, 47, 35, 44, 58, 25, 10, 5, 19, 48, 43, 31, 37, 6,
     21, 26, 32, 3, 15, 16, 22, 53, 38, 57, 63, 28, 60, 51, 9, 42]

Firstly, let's define necessary types like `Vector` and `Matrix`:

In [2]:
import itertools
from typing import Tuple, Sequence, Set, List, Dict, Optional

# vectors are represented as integers
Vector = int

# matrices are represented as tuple of row vectors
Matrix = Tuple[Vector, ...]

Here are some vector utilities:

In [3]:
# the dot product between 2 vectors: dot[u][v] = u.v
dot = [[bin(u & v).count('1') % 2 for u in range(64)] for v in range(64)]


def sum_vecs(vecs: Sequence[Vector]) -> Vector:
    """Add all vectors in `vecs`."""
    s = 0
    for v in vecs:
        s ^= v
    return s


def subspace(vecs: Sequence[Vector]) -> Set[Vector]:
    """Get all linear combinations of vectors in `vecs`."""
    result = {0}  # the zero vector is contained in every subspace
    for coeffs in itertools.product([0, 1], repeat=len(vecs)):
        result.add(sum_vecs([c * v for c, v in zip(coeffs, vecs)]))
    return result

This function extends an input matrix by adding a vector on top of it. Since we're building invertible matrices like $M_B$ and $M_A^{-1}$, their rows are always kept linearly independent. 

In [4]:
def extend(M: Matrix, M_times: Dict[Vector, Vector]) -> (
        List[Tuple[Matrix, Dict[Vector, Vector]]]):
    """Extend the input matrix `M` by putting a new vector `v` on top of `M`.
    `v` is guaranteed to be not in the subspace spanned by the row vectors in
    `M`. `M_times` is a mapping satisfying `M_times[v] = M*v` for `v` in F2^6.

    Returns all possible extensions of `M`.

    Note: The number of iterations in the main loop of this function is less
    than 64*64 (2**12).
    """
    result = []
    for r in set(range(64)).difference(subspace(M)):
        new_M = (r,) + M  # add new row on top of M
        new_M_times = {}
        for v in range(64):
            new_M_times[v] = (dot[r][v] << len(M)) + M_times[v]
        result.append((new_M, new_M_times))
    return result

This function decides whether or not a valid solution for $X_{2^i}$ has been found.

In [5]:
def verify(io_pairs: List[Tuple[int, int]], m: int) -> (
        Optional[Tuple[int, int]]):
    """Verify if a function given as a list of input/output pairs is
    well-defined and can be represented as: `f(x) = a*x + b mod m`. If 
    that is True, returns `a`, `b`."""
    f = {}
    for x, y in io_pairs:
        x, y = x % m, y % m
        if x in f:
            if f[x] != y:
                return None
        else:
            f[x] = y

    b = f[0]
    a = (f[1] - b) % m
    for x in range(2, m):
        if f[x] != (a * x + b) % m:
            return None

    return a, b

This function solves for all possible solutions of $X_{2^i}$ derived from a solution of $X_{2^{i-1}}$ given as input. A solution of $X_{2^i}$ consists of $X_{2^i}$, $M_{B[7-i:]}$ and $M_{A[7-i:]}^{-1}$.

In [6]:
Solution = Tuple[
    # Two numbers a, b representing `X_{2^i}`
    Tuple[int, int],

    # A submatrix of `Mb` and its corresponding mapping
    Tuple[Matrix, Dict[Vector, Vector]],

    # A submatrix of inverse of `Ma` and its corresponding mapping
    Tuple[Matrix, Dict[Vector, Vector]],
]


def solve_X2i(base: Solution) -> List[Solution]:
    """Solve for `X_{2^i}`, given a solution of `X_{2^{i-1}}` from which
    the new solutions are based on.
    
    Note: The number of iterations in the main loop of this function is less
    than 64*64*64 (2**18).
    """
    _, (Mb, Mb_times), (inv_Ma, inv_Ma_times) = base
    i = len(Mb) + 1
    m = 2 ** i
    result = []
    for new_Mb, new_Mb_times in extend(Mb, Mb_times):
        for new_inv_Ma, new_inv_Ma_times in extend(inv_Ma, inv_Ma_times):
            io_pairs = []
            for v in range(64):
                io_pairs.append((new_Mb_times[v], new_inv_Ma_times[S[v]]))
            a_and_b = verify(io_pairs, m)
            if a_and_b is not None:
                result.append((a_and_b, (new_Mb, new_Mb_times),
                               (new_inv_Ma, new_inv_Ma_times)))
    return result

Now, let's solve for each $X_{2^i}$:

In [7]:
%%time
NULL_MATRIX = tuple()
ZERO_MAP = {v: 0 for v in range(64)}
solutions = [((0, 0), (NULL_MATRIX, ZERO_MAP), (NULL_MATRIX, ZERO_MAP))]
for i in range(1, 7):
    new_solutions = []
    for sol in solutions:
        new_solutions += solve_X2i(sol)
    solutions = new_solutions
    print("Number of solutions for X_%d: %d" % (2 ** i, len(solutions)))

Number of solutions for X_2: 3
Number of solutions for X_4: 12
Number of solutions for X_8: 48
Number of solutions for X_16: 64
Number of solutions for X_32: 128
Number of solutions for X_64: 256
CPU times: user 10.4 s, sys: 3.59 ms, total: 10.4 s
Wall time: 10.4 s


So, there are 256 possible solutions. Let's print them all:

In [8]:
solutions.sort()
for (a, b), (Mb, _), (inv_Ma, _) in solutions:
    print(a, b, Mb, inv_Ma)

1 1 (41, 13, 44, 49, 45, 47) (43, 2, 53, 9, 37, 40)
1 1 (41, 13, 44, 49, 47, 45) (43, 2, 53, 9, 37, 13)
1 1 (41, 13, 44, 51, 45, 47) (43, 2, 53, 44, 37, 40)
1 1 (41, 13, 44, 51, 47, 45) (43, 2, 53, 44, 37, 13)
1 1 (41, 13, 46, 49, 45, 47) (43, 2, 16, 9, 37, 40)
1 1 (41, 13, 46, 49, 47, 45) (43, 2, 16, 9, 37, 13)
1 1 (41, 13, 46, 51, 45, 47) (43, 2, 16, 44, 37, 40)
1 1 (41, 13, 46, 51, 47, 45) (43, 2, 16, 44, 37, 13)
1 1 (41, 15, 44, 49, 45, 47) (43, 39, 53, 9, 37, 40)
1 1 (41, 15, 44, 49, 47, 45) (43, 39, 53, 9, 37, 13)
1 1 (41, 15, 44, 51, 45, 47) (43, 39, 53, 44, 37, 40)
1 1 (41, 15, 44, 51, 47, 45) (43, 39, 53, 44, 37, 13)
1 1 (41, 15, 46, 49, 45, 47) (43, 39, 16, 9, 37, 40)
1 1 (41, 15, 46, 49, 47, 45) (43, 39, 16, 9, 37, 13)
1 1 (41, 15, 46, 51, 45, 47) (43, 39, 16, 44, 37, 40)
1 1 (41, 15, 46, 51, 47, 45) (43, 39, 16, 44, 37, 13)
1 1 (43, 13, 44, 49, 45, 47) (14, 2, 53, 9, 37, 40)
1 1 (43, 13, 44, 49, 47, 45) (14, 2, 53, 9, 37, 13)
1 1 (43, 13, 44, 51, 45, 47) (14, 2, 53, 44, 37,

The simplest expression for $X$ can be seen is $X(y) = y + 1 \text{ mod } 64$. For each unique $X$, there are many choices for $B$ and $A$ which are given as $M_B$ and $M_A^{-1}$ respectively.